<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/PP/mean_prediction_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path
import plotly
import plotly.express as px
from glob import glob

In [2]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

In [3]:
path = Path("/content/drive/MyDrive/GSDC")
test_base = pd.read_csv(path / "baseline_locations_test.csv")
sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv", usecols=cols)
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

In [4]:
def mean_prediction(input_df):
    def make_lerp_data(input_df):
        org_colus = input_df.columns

        time_list = input_df[["collectionName", "millisSinceGpsEpoch"]].drop_duplicates()
        phone_list = input_df[["collectionName", "phoneName"]].drop_duplicates()
        tmp = time_list.merge(phone_list, on="collectionName", how="outer")

        output_df = tmp.merge(input_df, on=["collectionName", "millisSinceGpsEpoch", "phoneName"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        output_df = output_df.sort_values(["phone", "millisSinceGpsEpoch"])

        output_df["latDeg_pre"] = output_df["latDeg"].shift(1)
        output_df["latDeg_pro"] = output_df["latDeg"].shift(-1)
        output_df["lngDeg_pre"] = output_df["lngDeg"].shift(1)
        output_df["lngDeg_pro"] = output_df["lngDeg"].shift(-1)
        output_df["phone_pre"] = output_df["phone"].shift(1)
        output_df["phone_pro"] = output_df["phone"].shift(-1)
        output_df["millisSinceGpsEpoch_pre"] = output_df["millisSinceGpsEpoch"].shift(1)
        output_df["millisSinceGpsEpoch_pro"] = output_df["millisSinceGpsEpoch"].shift(-1)

        output_df = output_df[(output_df["latDeg"].isnull())&(output_df["phone"] == output_df["phone_pre"])&
                            (output_df["phone"] == output_df["phone_pro"])].copy()

        output_df["latDeg"] = output_df["latDeg_pre"] + ((output_df["latDeg_pro"] - output_df["latDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        output_df["lngDeg"] = output_df["lngDeg_pre"] + ((output_df["lngDeg_pro"] - output_df["lngDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        
        output_df = output_df[~output_df['latDeg'].isnull()]

        return output_df[org_colus]

    
    def calc_mean_pred(input_df, lerp_df):
        input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
        add_lerp = pd.concat([input_df, lerp_df])
        mean_pred_result = add_lerp.groupby(["collectionName", "millisSinceGpsEpoch"])[["latDeg", "lngDeg"]].mean().reset_index()
        output_df = input_df[["collectionName", "phoneName", "millisSinceGpsEpoch"]].copy()
        output_df = output_df.merge(mean_pred_result[["collectionName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]],
                                        on=["collectionName", "millisSinceGpsEpoch"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        return output_df


    test_lerp = make_lerp_data(input_df)
    test_mean_pred  = calc_mean_pred(input_df, test_lerp)

    sub["latDeg"] = test_mean_pred["latDeg"]
    sub["lngDeg"] = test_mean_pred["lngDeg"]

    output_df = sub.copy()

    return output_df

In [5]:
mean_prediction(test_base)

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416591,-122.082069
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416617,-122.082056
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416612,-122.082055
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416592,-122.082068
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416618,-122.082057
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334539,-121.899383
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334545,-121.899380
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334551,-121.899371
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334540,-121.899371
